In [ ]:
# Import necessary libraries for data processing, NLP, and API access
import pandas as pd
import nltk
from nltk.corpus import stopwords
import regex as re
import spacy
from transformers import pipeline
from tqdm.auto import tqdm
from openai import OpenAI

In [ ]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
## Loading Fitbit review dataset into a pandas DataFrame
df = pd.read_csv('C:/Users/DELL/Downloads/FitbitReviews/fitbit.csv')

In [ ]:
df.head()

,Unnamed: 0,source,review_id,user_name,review_title,review_description,rating,thumbs_up,review_date
0,0,Google Play,b52e6513-a955-438a-a048-4ae5e1e81abb,Marlena H.,NaN,Horrible change not allowing group challenges,2,0,2023-04-17 15:55:00
1,1,Google Play,d88c0a81-2611-4f53-b0c4-8f1a2442b6ac,Jeremy Chandler,NaN,Google already starting to sunset features and...,2,0,2023-04-17 15:39:49
2,2,Google Play,22205257-f36d-464d-ba47-4ef7283071a3,Stu McConnell,NaN,Whatever you have done with the latest update ...,1,0,2023-04-17 15:15:48
3,3,Google Play,c975c4ab-1a72-4e0f-a579-021ffbbe2d50,Emer Waters,NaN,"Challenges with friends/family were removed, s...",1,0,2023-04-17 14:59:02
4,4,Google Play,17ae54ba-9a79-4ba4-9ad2-a0b1020e6290,erica potts,NaN,Helpful for me to keep track of my steps,5,0,2023-04-17 14:51:12


In [ ]:
#drop unnecessary column or rows
df.drop(['source','review_title','review_date'],axis = 1, inplace = True)

In [ ]:
df.head()

,Unnamed: 0,review_id,user_name,review_description,rating,thumbs_up
0,0,b52e6513-a955-438a-a048-4ae5e1e81abb,Marlena H.,Horrible change not allowing group challenges,2,0
1,1,d88c0a81-2611-4f53-b0c4-8f1a2442b6ac,Jeremy Chandler,Google already starting to sunset features and...,2,0
2,2,22205257-f36d-464d-ba47-4ef7283071a3,Stu McConnell,Whatever you have done with the latest update ...,1,0
3,3,c975c4ab-1a72-4e0f-a579-021ffbbe2d50,Emer Waters,"Challenges with friends/family were removed, s...",1,0
4,4,17ae54ba-9a79-4ba4-9ad2-a0b1020e6290,erica potts,Helpful for me to keep track of my steps,5,0


In [ ]:
df.info() #checking the datatypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333491 entries, 0 to 333490
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Unnamed: 0          333491 non-null  int64 
 1   review_id           333491 non-null  object
 2   user_name           333491 non-null  object
 3   review_description  333353 non-null  object
 4   rating              333491 non-null  int64 
 5   thumbs_up           333491 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 15.3+ MB


In [ ]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
df['review_id'].nunique() #getting all the unique values

333491

In [ ]:
# Filter out 5-star ratings to focus on critical feedback
df_filtered = df[df['rating']!=5]

In [ ]:
df_filtered.head()

,review_id,user_name,review_description,rating,thumbs_up
0,b52e6513-a955-438a-a048-4ae5e1e81abb,Marlena H.,Horrible change not allowing group challenges,2,0
1,d88c0a81-2611-4f53-b0c4-8f1a2442b6ac,Jeremy Chandler,Google already starting to sunset features and...,2,0
2,22205257-f36d-464d-ba47-4ef7283071a3,Stu McConnell,Whatever you have done with the latest update ...,1,0
3,c975c4ab-1a72-4e0f-a579-021ffbbe2d50,Emer Waters,"Challenges with friends/family were removed, s...",1,0
5,33504376-5d25-4021-b1ba-d04718e5a1eb,Hitesh Sharma,"Watch faces not visible in the app, always sho...",1,0


In [ ]:
df_filtered.isnull().sum() #checking any null values

review_id             0
user_name             0
review_description    0
rating                0
thumbs_up             0
dtype: int64

In [ ]:
df_filtered.dropna(inplace = True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_3548\2582415459.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.dropna(inplace = True)


In [ ]:
df_filtered.shape

(203636, 5)

In [ ]:
df_processed = df_filtered[df_filtered['review_description'].str.split().str.len() > 5]

In [ ]:
df_processed.shape

(178995, 5)

In [ ]:
df_processed.to_csv('FitBitReviews_updated.csv',index = False)

In [ ]:
df_filtered = pd.read_csv(r"C:\Users\DELL\FitBitReviews_updated.csv")

In [ ]:
df_filtered.head()

,review_id,user_name,review_description,rating,thumbs_up
0,b52e6513-a955-438a-a048-4ae5e1e81abb,Marlena H.,Horrible change not allowing group challenges,2,0
1,d88c0a81-2611-4f53-b0c4-8f1a2442b6ac,Jeremy Chandler,Google already starting to sunset features and...,2,0
2,22205257-f36d-464d-ba47-4ef7283071a3,Stu McConnell,Whatever you have done with the latest update ...,1,0
3,c975c4ab-1a72-4e0f-a579-021ffbbe2d50,Emer Waters,"Challenges with friends/family were removed, s...",1,0
4,33504376-5d25-4021-b1ba-d04718e5a1eb,Hitesh Sharma,"Watch faces not visible in the app, always sho...",1,0


In [ ]:
df_filtered.describe()

,rating,thumbs_up
count,178995.000000,178995.000000
mean,2.146311,2.086215
std,1.187114,17.718597
min,1.000000,0.000000
25%,1.000000,0.000000
50%,2.000000,0.000000
75%,3.000000,1.000000
max,4.000000,1303.000000


In [ ]:
#Preprocessing and cleaning the Reviews


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Text cleaning function
def clean_review(text):
    text = str(text).lower()  # Lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation/numbers
    tokens = text.split()  # Tokenize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)  # Join back to string

# Apply cleaning to the review column
df_filtered['cleaned_review'] = df_filtered['review_description'].apply(clean_review)

# Remove reviews with 5 or fewer words
df_filtered = df_filtered[df_filtered['cleaned_review'].str.split().str.len() > 5].reset_index(drop=True)

# Show a sample of cleaned reviews
df_filtered[['review_description', 'cleaned_review']].head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,review_description,cleaned_review
0,Google already starting to sunset features and...,google already starting sunset features replac...
1,Whatever you have done with the latest update ...,whatever done latest update ruined things yet ...
2,"Challenges with friends/family were removed, s...",challenges friendsfamily removed fitbit pretty...
3,"Watch faces not visible in the app, always sho...",watch faces visible app always shows error oop...
4,This app used to be pretty good and I always p...,app used pretty good always paid premium remov...


In [ ]:
nlp = spacy.load("en_core_web_sm")

# Function to extract product and organization names
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ["ORG", "PRODUCT"]]

# Apply on the 'cleaned_review' column
df_filtered['entities'] = df_filtered['cleaned_review'].apply(extract_entities)

# Filter for reviews with extracted entities
entity_df = df_filtered[df_filtered['entities'].map(len) > 0]

# Preview top 5 results
print(entity_df[['cleaned_review', 'entities']].head())

                                       cleaned_review  \
0   google already starting sunset features replac...   
22  since google gotten hands app removed fun comm...   
24  android logging food recent frequent tabs list...   
26  tap gallery button get oops looks like theres ...   
73  rubbish owned google theyve removed challenges...   

                       entities  
0               [(google, ORG)]  
22          [(google, PRODUCT)]  
24             [(android, ORG)]  
26  [(tap gallery button, ORG)]  
73              [(google, ORG)]  


In [ ]:
df_filtered.head()

,review_id,user_name,review_description,rating,thumbs_up,cleaned_review,entities
0,d88c0a81-2611-4f53-b0c4-8f1a2442b6ac,Jeremy Chandler,Google already starting to sunset features and...,2,0,google already starting sunset features replac...,"[(google, ORG)]"
1,22205257-f36d-464d-ba47-4ef7283071a3,Stu McConnell,Whatever you have done with the latest update ...,1,0,whatever done latest update ruined things yet ...,[]
2,c975c4ab-1a72-4e0f-a579-021ffbbe2d50,Emer Waters,"Challenges with friends/family were removed, s...",1,0,challenges friendsfamily removed fitbit pretty...,[]
3,33504376-5d25-4021-b1ba-d04718e5a1eb,Hitesh Sharma,"Watch faces not visible in the app, always sho...",1,0,watch faces visible app always shows error oop...,[]
4,8f836e12-5adf-46a3-8d1f-e9c1d159d75c,Tegan Laing,This app used to be pretty good and I always p...,1,0,app used pretty good always paid premium remov...,[]


In [ ]:
df_filtered.to_csv('NER_Reviews.csv',index = False)

In [ ]:
df_filtered['ner_entities'].value_counts()

ner_entities
[]                  156846
[(x, ORG)]              21
[(x, MISC)]             19
[(al, ORG)]             10
[(##idas, MISC)]        10
                     ...  
[(##ness, ORG)]          1
[(mac, MISC)]            1
[(##unt, ORG)]           1
[(##ca, ORG)]            1
[(##pal, ORG)]           1
Name: count, Length: 72, dtype: int64

In [ ]:
df_filtered.to_csv('fitbit_ner.csv',index = False)

In [ ]:

tqdm.pandas()
# Load English model (make sure it's downloaded first with: python -m spacy download en_core_web_sm)
nlp = spacy.load("en_core_web_sm")
custom_entities = [
    "Fitbit", "Fitbit Charge 5", "Versa 2", "Apple Watch", "Mi Band 6", "Garmin",
    "Galaxy Watch", "Inspire HR", "Realme Watch", "Honor Band", "Noise Colorfit",'Google','Inspire','Sense'
]

# Keyword matcher function
def keyword_match(doc):
    matches = []
    for entity in custom_entities:
        if entity.lower() in doc.text.lower():
            matches.append(entity)
    return list(set(matches))

# Combine spaCy NER + keyword match
def extract_entities(text):
    doc = nlp(text)
    ner_entities = [ent.text for ent in doc.ents if ent.label_ in ['ORG', 'PRODUCT']]
    keyword_entities = keyword_match(doc)
    return list(set(ner_entities + keyword_entities))

# Sample data


df_filtered['extracted_entities'] = df_filtered['cleaned_review'].apply(extract_entities)

print(df_filtered)

                                   review_id        user_name  \
0       d88c0a81-2611-4f53-b0c4-8f1a2442b6ac  Jeremy Chandler   
1       22205257-f36d-464d-ba47-4ef7283071a3    Stu McConnell   
2       c975c4ab-1a72-4e0f-a579-021ffbbe2d50      Emer Waters   
3       33504376-5d25-4021-b1ba-d04718e5a1eb    Hitesh Sharma   
4       8f836e12-5adf-46a3-8d1f-e9c1d159d75c      Tegan Laing   
...                                      ...              ...   
157012  efef0980-97c8-4766-879c-7405e7a6583a    A Google user   
157013  fc25ff0b-8f0e-43fb-a8b0-2ce993ff1565    A Google user   
157014  0c8167e9-537e-408d-a34c-0d03e68a5ef2    A Google user   
157015  69d57cdc-168a-44c8-9d78-88e6b81dd283    A Google user   
157016  d068a07e-1c40-468f-b7e1-9597f4684ff8    A Google user   

                                       review_description  rating  thumbs_up  \
0       Google already starting to sunset features and...       2          0   
1       Whatever you have done with the latest update ...  

In [ ]:
!pip install vader

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
   ------ --------------------------------- 0.8/4.8 MB 6.7 MB/s eta 0:00:01
   ---------- ----------------------------- 1.3/4.8 MB 3.1 MB/s eta 0:00:02
   ----------------- ---------------------- 2.1/4.8 MB 3.9 MB/s eta 0:00:01
   ----------------- ---------------------- 2.1/4.8 MB 3.9 MB/s eta 0:00:01
   ------------------- -------------------- 2.4/4.8 MB 2.3 MB/s eta 0:00:02
   -------------------------- ------------- 3.1/4.8 MB 2.6 MB/s eta 0:00:01
   -------------------------- ------------- 3.1/4.8 MB 2.6 MB/s eta


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install vaderSentiment

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
df_filtered.to_csv('FitBit_NER_updated.csv', index = False)

In [ ]:
df_new = pd.read_csv(r"C:\Users\DELL\FitBit_NER_updated.csv")

In [ ]:
#Generating Sentiment score
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_sentiment_score(text):
    score = analyzer.polarity_scores(text)
    return score['compound']  # ranges from -1 (negative) to +1 (positive)

# Apply with progress bar
from tqdm import tqdm
tqdm.pandas()

df_new['sentiment_score'] = df_new['cleaned_review'].progress_apply(get_sentiment_score)

100%|████████████████████████████████████████████████████████████████████████| 157017/157017 [00:18<00:00, 8536.49it/s]


In [ ]:
df_new.head()

,review_id,user_name,review_description,rating,thumbs_up,cleaned_review,extracted_entities,sentiment_score
0,d88c0a81-2611-4f53-b0c4-8f1a2442b6ac,Jeremy Chandler,Google already starting to sunset features and...,2,0,google already starting sunset features replac...,"['Apple Watch', 'Google', 'google']",-0.9022
1,22205257-f36d-464d-ba47-4ef7283071a3,Stu McConnell,Whatever you have done with the latest update ...,1,0,whatever done latest update ruined things yet ...,[],-0.4215
2,c975c4ab-1a72-4e0f-a579-021ffbbe2d50,Emer Waters,"Challenges with friends/family were removed, s...",1,0,challenges friendsfamily removed fitbit pretty...,['Fitbit'],0.1779
3,33504376-5d25-4021-b1ba-d04718e5a1eb,Hitesh Sharma,"Watch faces not visible in the app, always sho...",1,0,watch faces visible app always shows error oop...,[],-0.0516
4,8f836e12-5adf-46a3-8d1f-e9c1d159d75c,Tegan Laing,This app used to be pretty good and I always p...,1,0,app used pretty good always paid premium remov...,['Fitbit'],0.8934


In [ ]:
df_new['user_name'].nunique()

55083

In [ ]:
df_new['user_name'].value_counts()

user_name
A Google user       99974
M                      12
Sarah                  11
Michelle               11
J                      11
                    ...  
Necole Seawright        1
Therese Sabarre         1
Angie Z                 1
Amy Whitaker            1
Hitesh Sharma           1
Name: count, Length: 55083, dtype: int64

In [ ]:
df_max_rated = df_new[(df_new['thumbs_up'] >= 10) & (df_new['rating'] == 4)]

In [ ]:
df_max_rated['review_description'].count()

np.int64(443)

In [ ]:
all_reviews = "\n".join(df_max_rated['review_description'].tolist())

In [ ]:
client = OpenAI(api_key='XXXX')

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Your GPT-based function (already defined)
def extract_insights(summary_text):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an expert product analyst for Fitbit reviewing customer feedback."},
                {"role": "user", "content": f"""Given the summarized customer reviews below, extract the following:

1. 🐞 Most Common Issues in detail.
2. 🔧 Suggestions or Feedback for Improvement (e.g. battery life, sync issues)
3. 📊 Mentions or comparisons with Competitors (e.g. Apple Watch, Mi Band, Samsung watch etc.)



Summarized Reviews:
\"\"\"{summary_text}\"\"\""""}
            ],
            temperature=0.7,
            max_tokens=1500
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error during GPT insight extraction: {e}")
        return None


In [ ]:
summary_report = extract_insights(all_reviews)
print(summary_report)

Based on the summarized customer reviews, here's the extracted information:

### 1. 🧩 Feature Requests
- Dark theme option for the app.
- Ability to log non-stereotypical fitness activities (e.g., lawn mowing, dance).
- Improved food database with more searchable items and the ability to add meals/recipes.
- Option to view and edit historical data easily, especially in sleep tracking and food logging.
- Automatic adjustment of daily goals based on user performance.
- Alarms for water intake reminders.
- Customizable alarms and notifications for specific activities.
- More dynamic use of app tiles to prioritize frequently used metrics.
- Ability to record multiple devices without having to re-pair each time.

### 2. 🔧 Suggestions or Feedback for Improvement
- Improve battery life, especially after updates.
- Fix syncing issues that require frequent reboots or app reinstallation.
- Ensure accurate heart rate and calorie tracking, especially during workouts.
- Allow automatic timezone upd

In [ ]:
df_min_rated = df_new[(df_new['thumbs_up'] >= 50) & (df_new['rating'] < 4)]

In [ ]:
all_reviews_min = "\n".join(df_min_rated['review_description'].tolist())

In [ ]:
summary_report_min = extract_insights(all_reviews_min)
print(summary_report_min)

Based on the summarized customer reviews, here is the requested information extracted:

1. **🐞 Most Common Issues in Detail:**
   - **Syncing Problems:** A significant number of users report that their devices fail to sync with the app, displaying messages like "Device not found" or "Cannot connect." This issue has persisted for many users over extended periods, with some stating that they have to reinstall the app or restart their devices frequently to briefly regain syncing capabilities.
   - **Bluetooth Connectivity Issues:** Users have noted problems with Bluetooth connections, where the app struggles to find the connected devices, causing frustration. Many have had to reset their devices or uninstall and reinstall the app multiple times.
   - **Time and Date Errors:** Some users report that their devices display incorrect time, which is not corrected without syncing, rendering the device almost useless.
   - **Notifications Failure:** Users have expressed dissatisfaction with the 